In [20]:
from common import *
import warnings
warnings.filterwarnings('ignore')
import operator

In [21]:
TEST = True; PROD = False

# -=#=-=#=-=#=-=#=-=#=- Features selection logic -=#=-=#=-=#=-=#=-=#=-=#=-

### -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=- Abstract Function datatype -=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-

In [22]:
from types import FunctionType

##### item checker

In [53]:
def item_check(a):
    if not isinstance(a, tuple):
        a = (a,)
    return a

#### Function super useful datatype

In [102]:
class Function(type):
    def __new__(self, *args, **kw):
        noattach = '_noattach_' in kw
        if len(args) == 1:
            if isinstance(args[0], __class__):
                return args[0]
            assert callable(args[0])
            if 'name' in kw:
                name = kw['name']
            else:
                name = args[0].__name__
            parents = ()
            attrs = {'__call__': args[0]}
            noattach = True
        else:
            name, parents, attrs = args
        for k, v in list(dict.items(attrs)):
            if k == '__getitem__':
                assert isinstance(v, FunctionType), "just for easier work on our side. it is a function(self, index)"
                if isinstance(v(self, 0), FunctionType): # if getitem returns a function
                    old_v = v
                    v = lambda self, a: __class__(old_v(self, item_check(a)))
            if not isinstance(v, classmethod) and callable(v) and not isinstance(v, __class__) and not noattach:
                attrs[k] = classmethod(v)
            
        return super().__new__(self, name, parents, attrs)
    
    def __init__(self, *args, **kw):
        #SEE('init', args, kw)
        if len(args) == 1:
            return
        name, parents, attrs = args
        for k in attrs:
            v = getattr(self, k)
            #SEE('init ->', k, v)
            if k[:1] != '_' and callable(v):
                setattr(self, k, __class__(v, _noattach_=True, name=k))
        
    def __call__(self, *a, **k):
        assert '__call__' in self.__dict__, f'__call__ not defined for Function object with name "{self.__name__}"'
        return self.__call__(*a, **k)
        
    def __getitem__(self, *a, **k):
        assert '__getitem__' in self.__dict__, f'__getitem__ not defined for Function object with name "{self.__name__}"'
        return self.__getitem__(*a, **k)
    
    __invert__ = lambda f: __class__(lambda *a, **k: ~f(*a,**k))
    __and__ = lambda f, g: __class__(lambda *a, **k: f(*a,**k) & g(*a,**k))
    __or__ = lambda f, g: __class__(lambda *a, **k: f(*a,**k) | g(*a,**k))
    __xor__ = lambda f, g: __class__(lambda *a, **k: f(*a,**k) ^ g(*a,**k))

In [103]:
# testing
if TEST:
    class What(metaclass=Function):
        def __call__(self, x, y):
            return np.sqrt(x**2 + y**2)
        def hi(self, *s):
            pass
            #SEE('hi, ' + s[0])
        __getitem__ = lambda self, a: lambda arg: (SEE('this is getitem', arg, arg*a), 'answer')[1]
    What.hi('u')
    What.hi#.__call__
    /SEE What[5].__class__
    /SEE What[5](6)

##### item getter

In [104]:
class Itemgetter():
    def __init__(self, func):
        self._func = Function(func)
    def __getitem__(self, a):
        return self._func(item_check(a))

In [105]:
# testing
if TEST:
    guy = Itemgetter(lambda *a: SEE(*a))
    guy[5.7,8]

In [106]:
# testing yo!
if TEST:
    class Yo():
        def __call__(self, x):
            SEE(x, '!')
    yo = Yo()
    yo.__call__ = lambda x: SEE('fuck', x)
    yo(5)

#### Function query language implementation

In [175]:
def scan(s):
    s = s.replace(' ','').replace('\n','').replace('\t','').replace('\r','') + '\0'
    i0, i = 0, 0
    while i0 < len(s)-1 and i < len(s):
        c = s[i]
        if 'a' <= c <= 'z' or 'A' <= c <= 'Z' or '0' <= c <= '9' or c == '_':
            i += 1
            continue
        elif c == '\0':
            yield s[i0:i]
            break
        elif c in '.~&|^()':
            if i0 < i:
                yield s[i0:i]
            yield c
            i0 = i = i+1
        elif c in '[{<':
            if i0 < i:
                yield s[i0:i]
            i0 = i
            op = c
            cl = {'[':']','{':'}','<':'>'}[op]
            opened = 1
            while opened > 0:
                i += 1
                opened += 1 if s[i] == op else -1 if s[i] == cl else 0
            i += 1
            if op != '<': # if this is not a comment
                yield s[i0:i]
            i0 = i
        else:
            raise AssertionError('bad syntax in input to scanner')

            
Token = namedtuple('Token', 'type value')

def tokenize(s):
    prevIsField = False
    for w in s:
        a = w[0]
        if w == '.':
            yield Token('self', '')
        elif 'a' <= a <= 'z' or 'A' <= a <= 'Z' or '0' <= a <= '9' or a in '_':
            yield Token('field', w)
            prevIsField = True
            continue
        elif w in ['&','^','|']:
            yield Token('op2', w)
        elif w in ['~']:
            yield Token('op1', w)
        elif w in ['(']:
            yield Token('_group', w)
        elif w in [')']:
            yield Token('group_', w)
        elif a == '{':
            assert prevIsField, '{} section must have been preceded by a field name'
            yield Token('op2', 'query')
            yield Token('raw', w)
        elif a == '[':
            if not prevIsField:
                yield Token('self', '')
            yield Token('op2', 'getitem')
            yield Token('raw', w)
        else:
            assert False, f'invalid word encountered in tokenizer input: {w}'
        prevIsField = False
        
class TokenInfo(O()):
    self = 'self, the owning Function instance'
    field = 'field name of Function instance'
    op2 = 'binary operator (look in the map)'
    op1 = 'unary operator (look in the map)'
    _group = "open grouping (just '(')"
    group_ = "close grouping (just ')')"
    raw = "raw data to be fed to operator"


def parse(s):
    stack = []
    for t in s:
        if t.type in ['field', 'self', 'raw']:
            yield t
        elif t.type in ['op2', 'op1']:
            while stack and stack[-1].type in ['op2', 'op1'] and parse.order[t.value] >= parse.order[stack[-1].value]:
                yield stack.pop()
            stack.append(t)
        elif t.type == '_group':
            stack.append(t)
        elif t.type == 'group_':
            while True:
                if not stack:
                    raise AssertionError('too many close parentheses')
                x = stack.pop()
                if x.type == '_group':
                    break
                else:
                    assert x.type in ['op2', 'op1'], 'internal error'
                    yield x
    while stack:
        x = stack.pop()
        if x.type == '_group':
            raise AssertionError('too many open parentheses')
        assert x.type in ['op2', 'op1'], 'internal error'
        yield x
parse.order = {'query': 0, 'getitem': 0, '~': 1, '&': 2, '^': 3, '|': 4}

    
def query(self, qs):
    revpol = list(parse(tokenize(scan(qs))))
    stack = []
    BUG.stack = stack
    for t in revpol:
        if t.type == 'self':
            stack.append(self)
        elif t.type == 'field':
            stack.append(getattr(self, t.value))
        elif t.type == 'raw':
            stack.append(t.value)
        elif t.type == 'op1':
            a = stack.pop()
            x = query.opTbl[t.value](a)
            stack.append(x)
        elif t.type == 'op2':
            b, a = stack.pop(), stack.pop()
            x = query.opTbl[t.value](a, b)
            stack.append(x)
    return stack.pop()

class GetItemHelper(metaclass=Singleton):
    def __getitem__(self, a):
        return a

query.opTbl = {
    '~': operator.__invert__,
    '&': operator.__and__,
    '^': operator.__xor__,
    '|': operator.__or__,
    'query': lambda f, q: f.query(q[1:-1]), #TODO can only go one level deep
    'getitem': lambda f, a: f[eval(f'GetItemHelper()[{a[1:-1]}]')]
}

Function.query = query

In [176]:
# testing function query
if TEST:
    qs = 'what2{f_ck} & no_[5,62,6:6] <whatevercomment> |  [9] & ~(ho_10|[3,5:9])'
    scanned = list(scan(qs))
    assert (scanned == ['what2','{f_ck}','&','no_','[5,62,6:6]','|','[9]','&','~','(','ho_10','|','[3,5:9]',')']), (
            'Unit test fail for scanner')    
    tokens = list(tokenize(scanned)) #TODO make unit test
    parsed = list(parse(tokens)) # TODO make unit test
    SEE(qs)
    SEE('-'*99)
    SEE(scanned)
    SEE('-'*99)
    for line in tokens:
        SEE(line)
    SEE('-'*99)
    for line in parsed:
        SEE(line)

In [177]:
def stringify(self, name=None):
    name = self.__name__ if name is None else name
    cond = lambda field: not(field=='query' or field[:1]=='_' and field!='__getitem__')
    if name == '__getitem__':
        yield '[]'
        return
    yield name
    if not any(cond(f) for f in dir(self)):
        return
    yield '{'
    for field in self.__dict__:
        if not cond(field):
            continue
        value = getattr(self, field)
        yield from stringify(getattr(self, field), field)
        yield ','
    yield '}'
Function.stringify = stringify

## .!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!. The actual Features subsets definitions!!!! .!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.

In [191]:
def time_slice(e):
    start, stop = e.start if e.start is not None else 0, e.stop if e.stop is not None else 0
    assert start >= 0 and stop >= 0
    return max(start, stop), min(start, stop)

def digit_check(x, s):
    '''checks `s in x` but what follows first occurence of `s` isn't a digit char'''
    try:
        i = x.index(s)
        return i+len(s)>=len(x) or x[i+len(s)] in '_ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
    except ValueError:
        return False
    
def i2s(e):
    if isinstance(e, int):
        return str(e)
    elif isinstance(e, slice):
        return '({}-{})'.format(*time_slice(e))
    
def gen_getitem(*words):
    return lambda f, a: lambda x: any(digit_check(x, word+i2s(e)) for e in item_check(a) for word in words)

def FFF__returns_range(x, e):
    ss = 'oo aoo cc acc (oo-aoo) (cc-acc) (aoo-acc) it af roo'.split()
    if e == 1:
        ret = x in ss
        ret |= '(it,af,it{1},aoo,acc)' in x
        ret |= any(x[-len(s):]==s for s in ss)
        ret |= any(_digit_check(x, s) for s in ss)
        return ret
    elif e == 2 and '(it{1},af{1},it{2},aoo{1},acc{1})' in x:
        return True
    elif isinstance(e, int):
        suf = '{' + str(e) + '}'
        return any(x[-len(s+suf):] == s+suf for s in ss)
    elif isinstance(e, slice):
        lrg, sml = time_slice(e)
        ret = ('(it,af,it{1},aoo,acc)' not in x and '(it{1},af{1},it{2},aoo{1},acc{1})' not in x and
            '({}..{})'.format(sml,lrg-1) in x or '{{{}/{}}}'.format(sml,lrg-1) in x)
        suf1, suf2 = 'Since'+str(lrg), str(lrg)
        ret |= x[-len(suf1):] == suf1
        ret |= x[-len(suf2):] == suf2 and x[-len(suf2)-1] not in '0123456789'
        return ret
    return False
    
def FFF__returns_span(x, e):
    if isinstance(e, int):
        assert e > 0
    suf1, suf2 = 'Since'+str(e), str(e)
    if x[-len(suf1):] == suf1 or x[-len(suf2):] == suf2:
        return True
    if e == 1:
        return any(FFF__returns_range(x, r) for r in [0,1,2,3,4,5])
    elif e == 2:
        return (x[-5:] == '{0/1}' or x[-5:] == '{1/2}') and ',' not in x
    elif e == 3:
        return x[-5:] == '{0/2}' and ',' not in x
    elif e == 5:
        return '(0..4)' in x or '(5..9)' in x or '(10..14)' in x or '(15..19)' in x
    elif e == 10:
        return '(0..9)' in x or '(5..14)' in x or '(10..19)' in x
    elif e == 15:
        return '(0..14)' in x or '(5..19)' in x
    elif e == 20:
        return '(0..19)' in x
    elif isinstance(e, slice):
        lrg, sml = time_slice(e)
        for i in [1,2,3,4,5,10,15,20,21,40,60,62,120,125,250]:
            if sml < i <= lrg and FFF__returns_span(x, i):
                return True
    return False
    

class FFF(metaclass=Function):
    # BIG NOTE item_check isn't necessary for the __getitem__ definitions anymore, it's done automatically by metaclass
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ super basic ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    class AssetEnc(metaclass=Function):
        __call__ = lambda f, x: 'assetCode' in x or 'assetName' in x
        Ids = lambda f, x: x in ['assetCodeId', 'assetNameId']
        
        class RandMap(metaclass=Function):
            __call__ = lambda f, x: 'randMap32' in x
            squared = Itemgetter(lambda a: lambda x:
                any(x[::-1].find(('randMap32_' + str(e**2))[::-1]) == 0 for e in item_check(a)))
            __getitem__ = (lambda f, a: lambda x:
                any(x[::-1].find(('randMap32_' + str(e))[::-1]) == 0 for e in item_check(a)))
        
        WTFF = lambda f, x: 'WTFF' in x
        InUni = lambda f, x: 'inUniCount' in x
        
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Since and MinMax pools ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    class Since(metaclass=Function):
        __call__ = lambda f, x: 'Since' in x
        __getitem__ = lambda f, a: lambda x: any('Since'+str(e) in x for e in item_check(a))
        Max = lambda f, x: 'max' in x or 'Max' in x
        Min = lambda f, x: 'min' in x or 'Min' in x
        
    class FracRec(metaclass=Function):
        __call__ = lambda f, x: 'fracRecordedDays' in x
        __getitem__ = lambda f, a: lambda x: any(x == 'fracRecordedDaysSince'+str(e) for e in item_check(a))
        
    class Volatility(metaclass=Function):
        __call__ = lambda f, x: 'olatility' in x
        __getitem__ = gen_getitem('olatility')
    
    class Drawdown(metaclass=Function):
        __call__ = lambda f, x: 'rawdown' in x
        __getitem__ = gen_getitem('rawdown')
    
    class Drawup(metaclass=Function):
        __call__ = lambda f, x: 'rawup' in x
        __getitem__ = gen_getitem('rawup')
        
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Stock day by day values ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    class Index(metaclass=Function):
        __getitem__ = ( lambda f, a: lambda x: any(x[::-1].find(f'{{{e}}}'[::-1] if e!=0 else '')==0
            and not (e==0 and x[-1]=='}') for e in item_check(a)) )
    
    class Price(metaclass=Function):
        __call__ = lambda f, x: x in ['open', 'close']
        Open = lambda f, x: x == 'open'
        Close = lambda f, x: x == 'close'
        
    class VP(metaclass=Function):
        __call__ = lambda f, x: x[:2] == 'vp' or x[:4] == 'm_vp'
        __getitem__ = gen_getitem('vp')
        
    class Return(metaclass=Function):
        pure = lambda f, x: all(b not in x for b in
                                 ['inSince','axSince','rawdown','rawup','ecordedDays','ssetCode','olume','olatility'])
        __call__ = lambda f, x: any('oo'+str(i) in x or 'cc'+str(i) in x for i in chain(range(10),['(']))
        class oo(metaclass=Function):
            __call__ = lambda f, x: 'oo' in x and not any(['aoo' in x, '-oo' in x, 'oo-' in x, ',oo' in x, 'oo,' in x])
            __getitem__ = gen_getitem('oo')
        class aoo(metaclass=Function):
            __call__ = lambda f, x: 'aoo' in x and not any(['-aoo' in x, 'aoo-' in x, ',aoo' in x, 'aoo,' in x])
            __getitem__ = gen_getitem('aoo')
        class doo(metaclass=Function):
            __call__ = lambda f, x: '(oo-aoo)' in x
            __getitem__ = gen_getitem('(oo-aoo)')
        class dd(metaclass=Function):
            __call__ = lambda f, x: 'dd' in x
            __getitem__ = gen_getitem('dd')
        class vp(metaclass=Function):
            __call__ = lambda f, x: x[:2] == 'vp' or x[:4] == 'm_vp'
            __getitem__ = gen_getitem('vp')
        
        __getitem__ = gen_getitem('oo','aoo','(oo-aoo)','dd')
        
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ MARKET ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    class Time(metaclass=Function):
        __call__ = lambda f, x: x in ['dayOfYear', 'dayOfWeek']
        long = lambda f, x: x == 'dayOfYear'
        short = lambda f, x: x == 'dayOfWeek'
        
    class Market(metaclass=Function):
        __call__ = lambda f, x: 'm_' in x
        
    class Weight(metaclass=Function):
        __call__ = lambda f, x: 'wVP' in x
        
    ################################################### DIVID BETWEEN GOOD^ AND BADv #######################################
    
    class Classic(metaclass=Function):
        pure = lambda f, x: all(b not in x for b in
                                 ['inSince','axSince','rawdown','rawup','ecordedDays','assetCode','olume','olatility'])
        __call__ = lambda f, x: any(s in x for s in ['oo','cc']) or x[:2] in ['it', 'af']
        class oo(metaclass=Function):
            __call__ = lambda f, x: 'oo' in x and not any(['aoo' in x, '-oo' in x, 'oo-' in x, ',oo' in x, 'oo,' in x])
            TEN = lambda f, x: 'ooTEN' in x
        class aoo(metaclass=Function):
            __call__ = lambda f, x: 'aoo' in x and not any(['-aoo' in x, 'aoo-' in x, ',aoo' in x, 'aoo,' in x])
            TEN = lambda f, x: 'aooTEN' in x
        class cc(metaclass=Function):
            __call__ = lambda f, x: 'cc' in x and not any(['acc' in x, '-cc' in x, 'cc-' in x, ',cc' in x, 'cc,' in x])
            TEN = lambda f, x: 'ccTEN' in x
        class acc(metaclass=Function):
            __call__ = lambda f, x: 'acc' in x and not any(['-acc' in x, 'acc-' in x, ',acc' in x, 'acc,' in x])
            TEN = lambda f, x: 'accTEN' in x
        class doo(metaclass=Function):
            __call__ = lambda f, x: '(oo-aoo)' in x
        dcc = lambda f, x: '(cc-acc)' in x
        daoc = lambda f, x: '(aoo-acc)' in x
        it = lambda f, x: x[:3] in ['it', 'it{']
        af = lambda f, x: x[:3] in ['af', 'af{']
        mix = lambda f, x: x[:3] == '(it'
        combo = lambda f, x: '/' in x or '..' in x
        __getitem__ = lambda f, a: lambda x: any(FFF__returns_range(x, e) for e in item_check(a))
        span = Itemgetter(lambda a: lambda x: any(FFF__returns_span(x, e) for e in item_check(a)))
    
#     class Volume(metaclass=Function):
#         __call__ = lambda f, x: 'volume' in x or 'Volume' in x
#         class Ratio(metaclass=Function):
#             '''~Back2 ~Back2{1} ~5 ~10 ~10{5} ~(20-5)'''
#             __call__ = lambda f, x: 'olumeRatio' in x
#             __getitem__ = lambda f, a: lambda x: x in ['volumeRatio'+str(e) for e in item_check(a)]
#         #Log = lambda f, x: 'Log' in x
#         __getitem__ = lambda f, a: lambda x: any('olume'+str(e) in x or (e==1 and x=='volume') for e in item_check(a))
#         dd = lambda f, x: 'olume' in x and 'dd' in x

In [129]:
#pd.to_pickle((F,P), '/big/data/saves/train_5searchbase.32.pkl')
#F,P = pd.read_pickle('/big/data/saves/train_5searchbase.32.pkl')
pd.set_option('display.max_rows', 2000)

In [166]:
import json
#json.dump(list(F.columns), open('feats.json', 'w'))
feats = json.load(open(top_dir/'work/feats.json', 'r'))

In [152]:
''.join(list(FFF.stringify()))

'FFF{AssetEnc{Ids,RandMap{squared{},[],},WTFF,InUni,},Since{[],Max,Min,},FracRec{[],},Volatility{[],},Drawdown{[],},Drawup{[],},Index{[],},Price{Open,Close,},VP{[],},Return{pure,oo{[],},aoo{[],},doo{[],},dd{[],},[],},Time{long,short,},Market,Weight,Classic{pure,oo{TEN,},aoo{TEN,},cc{TEN,},acc{TEN,},doo,dcc,daoc,it,af,mix,combo,[],span{},},}'

In [132]:
'''FFF{<officially available functions>
Classic{
    pure,
    oo{TEN,},    aoo{TEN,},
    cc{TEN,},    acc{TEN,},
    doo,         dcc,           daoc,
    it,          af,
    mix,
    combo,
    [],
    span{[]},},

Return{
    pure<exclude Since things>,
    oo{[],},
    aoo{[],},
    doo{[],},
    dd{[],},
    [< 5,10,20,60,120, 10:5,15:10,20:15, 15:5,20:10, 20:5 >],},
Volatility{[<5,10,20,60>],< & Since & Draw(up/down) & Market & Weight >},

Index{[<{0..whatever}>]},
Since{[<21,62,125,250>],Max,Min,},
Drawdown{[<5,10,20,10:5,20:10>],},
Drawup{[<5,10,20,10:5,20:10>],},
Price{Open,Close,},
VP{[],},

Time{long,short,},
Market,
Weight,

FracRec{[<Since>],},
AssetEnc{
    Ids,
    InUni,
    RandMap{squared{[<1..10>]},[],},WTFF<x2>,},
}''';

In [ ]:
'''free form organization
LOCAL
short
    oo/doo/dcc/doc/vp1dd 3 +- combos
    it/af/rr {0}/{1}
    combos mix {0}/{1}
med/long
    oo/aoo/doo 5 combos
    vp1dd/vp5dd 5 combos
    oo/aoo/doo long terms 60/120
    vp 5/10/20/60
    volatility 5/10/20/60
    fracRec 21/62/125/250
    minmax pools
        since 21/62/125/250
    x
        drawdown 1/5/10/20/10:5/20:10
        drawup ...
        maxsince {0}/{5}/{10}/{20}
        minsince ...
    x
        oo/aoo
        oo10/aoo10
        vp1dd/vp5dd/vp10dd
        volatility 10/20
    
GLOBAL
time short/long
    vp 1/5
    oo uniform/weight 1/5/10/20/60
    volatility 5/10/20/60
''';

In [206]:
fffList = [
    FFF.query('''<just a reference>
        <>~Market & (
            <>VP[5]
            | Return{pure} & Return{
                (oo | aoo) & ( [0,10:,20:,60:,120:] )
            }|Return & Drawdown[10,20:10] & Since[21,62,250] | Since{Min & [21]}
            | Return{dd & [5]} & Drawup[10,20:10] & Since[21,62,250]
        )|Market & Weight & (
            <>Return{ pure & oo & [10:,20:] } | Volatility[10,20]
        )|Time
        | AssetEnc{InUni} | FracRec{[62,250]} | VP<market&notmarket>
    '''),
    FFF.query('''<#F1 >
        <>~Market & Index[0] & (
            <>Return{pure} & Return{ (oo | aoo) & ( [5, 10, 15, 20, 60, 10:5, 15:10, 20:15, 15:5, 20:10, 20:5] ) }
            | Return{oo{.&[1]} | aoo[1]} & ( Drawdown[10] & Since[21,62] | Since{Min & [21]} )
            | VP[5] & ( Since{ Max & [21] } | Since{ Min & [21] })
        )|Market & Weight & Return{ [10, 60] }
    '''),
    FFF.query('''<#F2 >
        <>~Market & Index[0] & (
            <>Return{pure} & Return{ (aoo | doo | dd) & [5, 10, 15, 20, 60, 10:5, 15:10, 20:15, 15:5, 20:10, 20:5] }
            | Return{oo{.&[1]} | aoo[1]} & ( Drawdown[10,20:10] & Since[21,62,250] | Since{Min & [21,62]} )
            | VP[5,1] & ( Since{ Max & [21,62] } | Since{ Min & [21,62] })
        )|Market & Weight & Return{ [10, 60] }
        | Return{pure} & VP[10,20,60] | FracRec[21,60]
    '''),
    FFF.query('''<#F3 >
        <>~Market & Index[0] & (
            <>Return{pure} & Return{
                <>oo & [1, 5,10,15, 20,60, 10:5,15:10,20:15, 15:5,20:10, 20:5]
                | aoo & [1, 5,10,15, 20,60]
                | doo & [10:5, 15:10, 20:15, 15:5,20:10, 20:5]
                | vp & [10, 20, 60]
            }|Return{oo{.&[1]}} & ( Drawdown[10,20:10] & Since[21,62,250] | Since{Min & [21,62]} )
            | Return{oo{.&[10]}} & ( Drawdown[20] & Since[21,62,250] | Since{Min & [21,62]} )
            | VP[10] & ( Since{ Max & [21,62,250] } | Since{ Min & [21,62,250] })
        )|Market & Return{ [10, 60] }
        | Return{pure} & VP[60]
    '''),
    FFF.query('''<#F4 >
        <>~Market & Index[0] & (
            <>Return{pure} & Return{ (oo | aoo | vp[1]) & ( [5, 10, 60, 10:5, 15:10, 20:15, 15:5, 20:10] ) }
            | Return{oo{.&[1]} | aoo[1]} & ( Drawdown[10] & Since[21,62] | Since{Min & [21]} )
            | VP[1] & ( Since{ Max & [21,62,250] } | Since{ Min & [21,62,250] })
        )|Market & ( Weight & Return{ [20] } | Volatility[20] )
        | Time | FracRec[60,250]
    '''),
]

In [207]:
(lambda x: (print(len(x)), x)[1] )(list(filter(fffList[4], feats)))

41


['dayOfYear',
 'dayOfWeek',
 'oo5',
 'oo10',
 'oo(10-5)',
 'oo(15-10)',
 'oo(20-15)',
 'oo(15-5)',
 'oo(20-10)',
 'aoo5',
 'aoo10',
 'aoo(10-5)',
 'aoo(15-10)',
 'aoo(20-15)',
 'aoo(15-5)',
 'aoo(20-10)',
 'vp1dd5',
 'vp1dd10',
 'vp1dd(10-5)',
 'vp1dd(15-10)',
 'vp1dd(20-15)',
 'vp1dd(15-5)',
 'vp1dd(20-10)',
 'oo60',
 'aoo60',
 'oo1Drawdown10Since21',
 'oo1MinSince21',
 'aoo1Drawdown10Since21',
 'aoo1MinSince21',
 'vp1dd1MaxSince21',
 'vp1dd1MinSince21',
 'oo1Drawdown10Since62',
 'aoo1Drawdown10Since62',
 'vp1dd1MaxSince62',
 'vp1dd1MinSince62',
 'fracRecordedDaysSince250',
 'vp1dd1MaxSince250',
 'vp1dd1MinSince250',
 'wVPm_oo20',
 'm_volatility20',
 'wVPm_volatility20']